In [90]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [91]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input,decode_predictions
from tensorflow.keras.models import Model

In [92]:
train_dir='/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/train'
test_dir='/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/test'

In [93]:
Data_Gen=ImageDataGenerator(rescale=1/255,validation_split=0.25)

In [94]:
train_df=pd.read_csv('/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv')
test_df=pd.read_csv('/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/Testing_set.csv')

In [95]:
print(train_df.head())
print('\n')
test_df.head()

In [96]:
train_itt=Data_Gen.flow_from_dataframe(train_df,directory=train_dir,x_col='filename',
                                       y_col='label',target_size=(224, 224),batch_size=64,subset='training')
test_itt=Data_Gen.flow_from_dataframe(train_df,directory=train_dir,x_col='filename',
                                       y_col='label',target_size=(224, 224),batch_size=64,subset='validation')

In [97]:
model=ResNet50(include_top=False,input_shape=(224,224,3))

In [98]:
for layer in model.layers:
    layer.trainable=False

In [99]:
flatten_layer=Flatten()(model.output)

fC_layer=Dense(512,activation='relu')(flatten_layer)
fC_layer_2=Dense(256,activation='relu')(fC_layer)
fC_softmax_layer=Dense(15,activation='softmax')(fC_layer_2)

In [100]:
model= Model(inputs=model.input,outputs=fC_softmax_layer)

In [101]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_itt,validation_data=test_itt,epochs=35)

In [102]:
losses = pd.DataFrame(model.history.history)

In [103]:
plt.plot(losses['loss'])
plt.plot(losses['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('no of epochs')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [104]:
plt.plot(losses['accuracy'])
plt.plot(losses['val_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('no of epochs')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [105]:
class_label= test_itt.class_indices
class_label

In [106]:
model.save('VGG19_HAR_trained_model.h5')

In [107]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
def prediction(path):
    image=load_img(path,target_size=(224,224))
    plt.imshow(image)
    plt.show()
    image=img_to_array(image)
    image=image.reshape(1,224,224,3)
    result=model.predict(image)
    result=np.argmax(result)
    return print([key for key in class_label][result])

In [108]:
prediction('/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_5358.jpg')